In [4]:
import json
import opendssdirect as dss
import time
import math
import re

In [2]:
#open conflict matrix
fcm = open("ConflictMatrix.json", "r")
ConflictMatrix = json.loads(fcm.read())

#open dist_equip_map
fcm = open("AddressableEquipment.json", "r")
AddressableEquipment = json.loads(fcm.read())

DistributedConflictMatrix = {}
for area_id in list(AddressableEquipment.keys()):
    DistributedConflictMatrix[area_id] = {}
    for eq_id in AddressableEquipment[area_id]:
        DistributedConflictMatrix[area_id][eq_id] = ConflictMatrix['setpoints'][eq_id]
        
DistributedConflictMatrix

{'123.1': {'RatioTapChanger.reg4a': {'decarbonization-app': 3,
   'profit_cvr-app': 0,
   'resilience-app': 3},
  'RatioTapChanger.reg4b': {'decarbonization-app': 3,
   'profit_cvr-app': 0,
   'resilience-app': 5},
  'RatioTapChanger.reg4c': {'decarbonization-app': -9,
   'profit_cvr-app': 4,
   'resilience-app': -1},
  'BatteryUnit.battery4': {'decarbonization-app': -150000.0,
   'profit_cvr-app': -150000.0,
   'resilience-app': 150000.0}},
 '123.2': {'BatteryUnit.battery5': {'decarbonization-app': -250000.0,
   'profit_cvr-app': -221526.5,
   'resilience-app': 250000.0}},
 '123.3': {'BatteryUnit.battery3': {'decarbonization-app': -100000.0,
   'profit_cvr-app': -100000.0,
   'resilience-app': 100000.0}},
 '123.4': {'RatioTapChanger.reg1a': {'decarbonization-app': 6,
   'profit_cvr-app': -5,
   'resilience-app': 0},
  'RatioTapChanger.reg2a': {'decarbonization-app': -12,
   'profit_cvr-app': -1,
   'resilience-app': 2},
  'RatioTapChanger.reg3a': {'decarbonization-app': -12,
   'profi

In [3]:
# This should be part of init
dss.run_command('Redirect ./123Bus/Run_IEEE123Bus.dss')
dss.run_command('Compile ./123Bus/Run_IEEE123Bus.dss')
dss.Solution.Solve()

xfmr_id = dss.Transformers.First()
self.tap_vals = {}
while xfmr_id:
    tap_vals[dss.Transformers.Name()] = round((dss.Transformers.Tap()-1)*160)
    xfmr_id = dss.Transformers.Next()



NameError: name 'self' is not defined

In [4]:
# this should be used to eliminate alternatives

for setpoint in list(ConflictMatrix["setpoints"].keys()):
    if "Tap" in setpoint:
        eq_str = re.match(r'RatioTapChanger\.(.*)', setpoint)
        eq_name = eq_str.group(1)
        
        max_budget_tap = self.tap_vals[eq_name] + 2
        min_budget_tap = self.tap_vals[eq_name] - 2
        print(eq_name, max_budget_tap, min_budget_tap)   
        
# After resolution, we update self.tap_vals with final value

NameError: name 'self' is not defined

In [4]:
feeder_mrid = "_E3D03A27-B988-4D79-BFAB-F9D37FB289F7" #123 deconf
# feeder_mrid = "_C1C3E687-6FFD-C753-582B-632A27E28507"  # 123 bus
# feeder_mrid = "_49AD8E07-3BF9-A4E2-CB8F-C3722F837B62"  # 13 bus
# feeder_mrid = "_5B816B93-7A5F-B64C-8460-47C17D6E4B0F" # 13 bus asets
#feeder_mrid = "_4F76A5F9-271D-9EB8-5E31-AA362D86F2C3"  # 8500 node
#feeder_mrid = "_67AB291F-DCCD-31B7-B499-338206B9828F" # J1
#feeder_mrid = "_9CE150A8-8CC5-A0F9-B67E-BBD8C79D3095"  # R2 12.47 3
#feeder_mrid = "_EE71F6C9-56F0-4167-A14E-7F4C71F10EAA" #9500 node

In [10]:
import importlib
cim_profile = 'rc4_2021'
cim = importlib.import_module('cimgraph.data_profile.' + cim_profile)
sparql = importlib.import_module('cimgraph.loaders.sparql.' + cim_profile)

from cimgraph.loaders import Parameter, ConnectionParameters
from cimgraph.loaders.blazegraph.blazegraph import BlazegraphConnection
from cimgraph.loaders.gridappsd import GridappsdConnection, get_topology_response
from cimgraph.models import DistributedModel

# Blazegraph connection for running outside the container
params = ConnectionParameters([Parameter(key="url", value="http://localhost:8889/bigdata/namespace/kb/sparql")])
bg = BlazegraphConnection(params, 'rc4_2021')
topology_response = get_topology_response(feeder_mrid)

In [9]:
import os
from gridappsd import GridAPPSD
os.environ['GRIDAPPSD_APPLICATION_ID'] = 'gridappsd-topology-processor'
os.environ['GRIDAPPSD_APPLICATION_STATUS'] = 'STARTED'
os.environ['GRIDAPPSD_USER'] = 'app_user'
os.environ['GRIDAPPSD_PASSWORD'] = '1234App'
gapps = GridAPPSD()
assert gapps.connected

topic = "goss.gridappsd.request.data.topology"

message = {
   "requestType": "GET_SWITCH_AREAS",
   "modelID":  feeder_mrid,
   "resultFormat": "JSON"
}

dist_api_message = gapps.get_response(topic, message, timeout=60)

In [11]:
feeder = cim.Feeder(mRID=feeder_mrid)
network = DistributedModel(connection=bg, feeder=feeder, topology=topology_response['feeders'])

In [40]:
network_area = network.switch_areas[2]
for eq_type in list(network_area.typed_catalog.keys()):
    print(eq_type.__name__)
    for eq_id in list(network_area.typed_catalog[eq_type].keys()):
        name = network_area.typed_catalog[eq_type][eq_id].name
        print(name)

BatteryUnit
battery3
PowerElectronicsConnection
dg_48
battery3
dg_42
PhotovoltaicUnit
dg_48
dg_42
EnergyConsumer
s65c
s63a
s62c
s59b
s56b
s53a
s52a
s65b
s60a
s65a
s66c
s58b
s64b
s55a
Analog
ACLineSegment_l116_Voltage
LoadBreakSwitch_sw2_Voltage
LoadBreakSwitch_sw2_Voltage
LoadBreakSwitch_sw6_Voltage
LoadBreakSwitch_sw2_Voltage
LoadBreakSwitch_sw4_Voltage
LoadBreakSwitch_sw4_Voltage
LoadBreakSwitch_sw4_Voltage
ACLineSegment_l116_Voltage
ACLineSegment_l116_Voltage
LoadBreakSwitch_sw6_Voltage
ACLineSegment_l61_Voltage
ACLineSegment_l57_Voltage
ACLineSegment_l65_Voltage
ACLineSegment_l55_Voltage
ACLineSegment_l65_Voltage
ACLineSegment_l54_Voltage
ACLineSegment_l61_Voltage
ACLineSegment_l63_Voltage
ACLineSegment_l56_Voltage
ACLineSegment_l53_Voltage
ACLineSegment_l55_Voltage
ACLineSegment_l62_Voltage
ACLineSegment_l55_Voltage
ACLineSegment_l64_Voltage
ACLineSegment_l52_Voltage
ACLineSegment_l63_Voltage
ACLineSegment_l63_Voltage
ACLineSegment_l64_Voltage
ACLineSegment_l53_Voltage
ACLineSegme

In [50]:
network.switch_areas[8].typed_catalog[cim.TransformerTank]

IndexError: list index out of range

In [59]:
addressable_equip = {}
for index in range(len(network.switch_areas)):
    network_area = network.switch_areas[index-1]
    addressable_equip['123.'+str(index)] = []
    if cim.TransformerTank in network_area.typed_catalog.keys():
        eq_type = cim.TransformerTank
        for eq_id in list(network_area.typed_catalog[eq_type].keys()):
            name = network_area.typed_catalog[eq_type][eq_id].name
            addressable_equip['123.'+str(index)].append('RatioTapChanger.'+name)
    if cim.BatteryUnit in network_area.typed_catalog.keys():
        eq_type = cim.BatteryUnit
        for eq_id in list(network_area.typed_catalog[eq_type].keys()):
            name = network_area.typed_catalog[eq_type][eq_id].name
            addressable_equip['123.'+str(index)].append('BatteryUnit.'+name)
addressable_equip

{'123.0': ['RatioTapChanger.reg3a',
  'RatioTapChanger.reg3c',
  'RatioTapChanger.reg2a',
  'BatteryUnit.battery1'],
 '123.1': ['BatteryUnit.battery2'],
 '123.2': ['BatteryUnit.battery5'],
 '123.3': ['BatteryUnit.battery3'],
 '123.4': [],
 '123.5': [],
 '123.6': [],
 '123.7': ['RatioTapChanger.reg4a',
  'RatioTapChanger.reg4c',
  'RatioTapChanger.reg4b',
  'BatteryUnit.battery4']}

In [ ]:
node_list = []
for node in network.switch_areas[2].typed_catalog[cim.ConnectivityNode].values():
    node_list.append(node.name)

In [ ]:
node_list

In [ ]:
dist_eqip =



In [ ]:
alt = {}
alt['a1'] = {}
alt['a1']['RatioTapChanger:reg1a'] =  -2
alt['a1']['RatioTapChanger:reg2a'] =  -1
alt['a1']['RatioTapChanger:reg3a'] =  4

alt['a2'] = {}
alt['a2']['RatioTapChanger:reg1a'] =  -1
alt['a2']['RatioTapChanger:reg2a'] =  -1
alt['a2']['RatioTapChanger:reg3a'] =  4

alt['a3'] = {}
alt['a3']['RatioTapChanger:reg1a'] =  -0
alt['a3']['RatioTapChanger:reg2a'] =  -1
alt['a3']['RatioTapChanger:reg3a'] =  4


In [5]:
dss.run_command('Redirect ./123Bus/Run_IEEE123Bus.dss')

''

In [6]:
dss.run_command('Compile ./123Bus/Run_IEEE123Bus.dss')

''

In [7]:
dss.Solution.Solve()


In [8]:
from dss import DSS

In [9]:
DSS.NewContext()

In [8]:
StartTime = time.process_time()

violations = False

bus_volt = dss.Circuit.AllBusMagPu()
print(min(bus_volt), max(bus_volt))
if min(bus_volt) < 0.95:
    violations = True
    print("violations!")
elif max(bus_volt) > 1.10:
    violations = True
    print("violations!")

sub = dss.Circuit.TotalPower()
sub_kva = math.sqrt((sub[0]*sub[0]) + (sub[1]*sub[1]))
if sub_kva > 5000 or sub_kva < -5000:
    violations = True
    
dss.Circuit.SetActiveClass('Transformer')
device = dss.Circuit.FirstElement()
while device:
    a = dss.CktElement.CurrentsMagAng()
    if a[0] > 668:
        violations = True
        
    device = dss.Circuit.NextElement()
print(time.process_time() - StartTime)
    

0.9799452732352611 1.0574482418074136
0.0029417750000000353


In [12]:
StartTime = time.process_time()

c_t = 0.02817 #transmission price $/kWh
c_r = 0.1090 #retail rate
c_n = 0.01786 #feed-in tariff
c_dies = 0.34 #diesel fuel cost
c_lng = 0.25 #lng fuel cost

e_t = 1.205 #grid co2 lb/kWh
e_dies = 2.44
e_lng = 0.97


bus_shunt_p = {}
total_load = 0
total_pv = 0
total_dg = 0
profit = 0
emissions = 0

sub = dss.Circuit.TotalPower()
p_sub = sub[0]

loss = dss.Circuit.Losses()
p_loss = loss[0]/1000

dss.Circuit.SetActiveClass('Load')
device = dss.Circuit.FirstElement()

while device:
    output = dss.CktElement.TotalPowers()
    bus = dss.CktElement.BusNames()
    bus_shunt_p[bus[0]] = output[0]
    total_load = total_load + output[0]
    device = dss.Circuit.NextElement()
    
dss.Circuit.SetActiveClass('PVSystem')
device = dss.Circuit.FirstElement()
while device:
    output = dss.CktElement.TotalPowers()
    bus = dss.CktElement.BusNames()
    total_pv = total_pv + output[0]
    if bus[0] in bus_shunt_p.keys():
        bus_shunt_p[bus[0]] = bus_shunt_p[bus[0]] + output[0]
    else:
        bus_shunt_p[bus[0]] = output[0]
    device = dss.Circuit.NextElement()
    
dss.Circuit.SetActiveClass('Generator')
device = dss.Circuit.FirstElement()
while device:
    output = dss.CktElement.TotalPowers()
    p_dg = output[0]
    total_dg = total_dg + p_dg
    if 'dies' in dss.CktElement.Name():
        profit = profit + p_dg*c_dies
        emissions = emissions - p_dg*e_dies
    elif 'lng' in dss.CktElement.Name():
        profit = profit + p_dg*c_lng
        emissions = emissions - p_dg*e_lng
    device = dss.Circuit.NextElement()

emissions = emissions - p_sub*e_t
profit = profit + p_sub*c_t
for bus in list(bus_shunt_p.keys()):
    p = bus_shunt_p[bus]
    if  p > 0:
        profit = profit + p*c_r
    else:
        profit = profit + p*c_n
        

    
print(time.process_time() - StartTime)
print(profit)
print(p_loss)
print(emissions)

0.003859070999999936
68.79678324142444
60.286882884747556
829.8612139198714


In [ ]:
dss.Circuit.SetActiveClass('Storage')
device = dss.Circuit.FirstElement()
total_batt = 0
while device:
    output = dss.CktElement.TotalPowers()
    total_batt = total_batt + output[0]
    device = dss.Circuit.NextElement()
print(total_batt)

In [ ]:
max([-50, -100])

In [ ]:
bus_volt = dss.Circuit.AllBusMagPu()
print(min(bus_volt), max(bus_volt))
if min(bus_volt) < 0.95:
    print("violations!")
elif max(bus_volt) > 1.10:
    print("violations!")

In [ ]:
dss.Circuit.AllBusMagPu()

In [ ]:
dss.Transformers.AllNames()

In [ ]:
xfmr_id = dss.Transformers.First()
print(dss.Transformers.NumTaps())
print(dss.Transformers.MinTap())
print(dss.Transformers.MaxTap())

In [ ]:
dss.Circuit.Losses()

In [ ]:
dss.Circuit.TotalPower()

In [ ]:
dss.Capacitors.AllNames()

In [ ]:
cap_id = dss.Capacitors.First()
cap_vals = {}
while cap_id:
    cap_vals[dss.Capacitors.Name()] = dss.Capacitors.States()
    cap_id = dss.Capacitors.Next()

In [ ]:
dss.Capacitors.

In [ ]:
StartTime = time.process_time()
load_id = dss.Loads.First()
load_vals = {}
total_load = 0
while load_id:
    load = dss.Loads.kW()
    load_vals[dss.Loads.Name()] = load
    load_id = dss.Loads.Next()
    total_load = total_load + load
print(time.process_time() - StartTime)

In [ ]:
dss.Ckt.Name()

In [ ]:
dss.run_command('Export Loads')

In [ ]:
dss.Circuit.SetActiveClass('Capacitor')
device = dss.Circuit.FirstElement()
total_load = 0
bus_shunt_p = {}
while device:
    output = dss.CktElement.VoltagesMagAng()
    print(output)
#     bus = dss.CktElement.BusNames()
#     bus_shunt_p[bus[0]] = output[0]
    device = dss.Circuit.NextElement()
# print(bus_shunt_p)

In [ ]:
dss.Circuit.SetActiveClass('Transformer')

dss.CktElement.Name()

In [ ]:
dss.Circuit.SetActiveClass('Transformer')
device = dss.Circuit.FirstElement()
total_load = 0
bus_shunt_p = {}
while device:
    s = dss.CktElement.TotalPowers()
    a = dss.CktElement.CurrentsMagAng()
    print(a[0])
    kva = math.sqrt((s[0]*s[0]) + (s[1]*s[1]))
    print(dss.CktElement.Name())
#     print(kva)
#     bus = dss.CktElement.BusNames()
#     bus_shunt_p[bus[0]] = output[0]
    device = dss.Circuit.NextElement()
# print(bus_shunt_p)

In [ ]:
a = dss.Circuit.SetActiveElement('PVSystem.')

In [ ]:
dss.CktElement.BusNames(

In [ ]:
dss.dss_lib.ActiveClass_Get_First()

In [ ]:
(dss.dss_lib.Bus_Get_Next())
ffi.string(dss.dss_lib.Bus_Get_Name())

In [ ]:
tap = 1
val = 1 + tap/160
StartTime = time.process_time()
dss.run_command('BatchEdit PVsystem..* irradiance=1')
dss.run_command('Transformer.reg4a.Taps=[1.0 %s]'%val)
dss.Solution.SolveNoControl()
print(time.process_time() - StartTime)


In [11]:
dss.run_command("Storage.battery1.kW = 125.0")
dss.run_command("Storage.battery2.kW= 100")
dss.run_command("Storage.battery3.kW= 100")
dss.run_command("Storage.battery4.kW = 150")
dss.run_command("Storage.battery5.kW= -250")
dss.Solution.SolveNoControl()


In [ ]:
from opendssdirect.utils import Iterator
import cffi

In [ ]:
[i() for i in Iterator(dss.Bus, "name")]

In [ ]:
from cffi import FFI
ffi = FFI()

In [ ]:
a = dss.Bus.AllPCEatBus()

In [ ]:
dss.Circuit.SetActiveElement(a[0])

In [ ]:
sub = dss.Circuit.TotalPower()
math.sqrt((sub[0]*sub[0]) + (sub[1]*sub[1]))

In [ ]:
sub

In [ ]:
import math

In [10]:
import json
with open('resolutionResults.json', encoding='utf-8') as inputfile:
    resolutionResult = json.load(inputfile)

In [11]:
rheader = ['time','solver', 'failed', 'BatteryUnit.battery1', 'BatteryUnit.battery2', 'BatteryUnit.battery3',
   'BatteryUnit.battery4',
   'BatteryUnit.battery5',
   'RatioTapChanger.reg1a',
   'RatioTapChanger.reg2a',
   'RatioTapChanger.reg3a',
   'RatioTapChanger.reg3c',
   'RatioTapChanger.reg4a',
   'RatioTapChanger.reg4c']

with open('results.csv', 'w') as csv_file:
    r = ",".join(rheader)
    csv_file.write(f"{r}\n")

In [12]:
for time in list(resolutionResult.keys()):
    row = ['']*14

    row[0] = str(time)
    row[1] = str(resolutionResult[time]['deconflictTime'])
    row[2] = str(resolutionResult[time]['deconflictionFailed'])
    for index in range(3, len(rheader)):
        if rheader[index] in resolutionResult[time]['setpoints']:
            row[index] = str(resolutionResult[time]['setpoints'][rheader[index]])
    with open('results.csv', 'a') as csv_file:
        r = ",".join(row)
        csv_file.write(f"{r}\n")

In [13]:
with open('../conflictResults.json', encoding='utf-8') as inputfile:
    conflictResult = json.load(inputfile)

In [14]:
header = ['time',
          'BatteryUnit.battery1',"decarbonization-app", "profit_cvr-app", "resilience-app",
          'BatteryUnit.battery2',"decarbonization-app", "profit_cvr-app", "resilience-app",
          'BatteryUnit.battery3',"decarbonization-app", "profit_cvr-app", "resilience-app",
   'BatteryUnit.battery4',"decarbonization-app", "profit_cvr-app", "resilience-app",
   'BatteryUnit.battery5',"decarbonization-app", "profit_cvr-app", "resilience-app",
   'RatioTapChanger.reg1a',"decarbonization-app", "profit_cvr-app", "resilience-app",
   'RatioTapChanger.reg2a',"decarbonization-app", "profit_cvr-app", "resilience-app",
   'RatioTapChanger.reg3a',"decarbonization-app", "profit_cvr-app", "resilience-app",
   'RatioTapChanger.reg3c',"decarbonization-app", "profit_cvr-app", "resilience-app",
   'RatioTapChanger.reg4a',"decarbonization-app", "profit_cvr-app", "resilience-app",
   'RatioTapChanger.reg4c', "decarbonization-app", "profit_cvr-app", "resilience-app"]

In [15]:
with open('conflicts.csv', 'w') as csv_file:
    r = ",".join(header)
    csv_file.write(f"{r}\n")

In [16]:
for time in list(conflictResult.keys()):
    row = ['']*45

    row[0] = str(time)
    for index in range(1, len(header),4):
        if header[index] in conflictResult[time]['setpoints']:
            row[index] = str(resolutionResult[time]['setpoints'][rheader[(round(index/4)+3)]])
            row[index+1] = str(conflictResult[time]['setpoints'][header[index]][header[index+1]])
            row[index+2] = str(conflictResult[time]['setpoints'][header[index]][header[index+2]])
            row[index+3] = str(conflictResult[time]['setpoints'][header[index]][header[index+3]])
            
    with open('conflicts.csv', 'a') as csv_file:
        r = ",".join(row)
        csv_file.write(f"{r}\n")

In [17]:
cheader = ['time', 'profit', 'losses', 'emissions', 'battery charging']

with open('criteriaResults.json', encoding='utf-8') as inputfile:
    criteriaResult = json.load(inputfile)
    
with open('criteria.csv', 'w') as crit_file:
    r = ",".join(cheader)
    crit_file.write(f"{r}\n")

In [18]:
for time in list(criteriaResult):
    row = [str(time)]
    row.append(str(criteriaResult[str(time)]))
    with open('criteria.csv', 'a') as csv_file:
        r = ",".join(row)
        csv_file.write(f"{r}\n")

In [19]:
criteriaResult

{'1': [135.0159597637487,
  9.70686320826802,
  687.981632232939,
  -824.9999987875345],
 '2': [135.44983869179543,
  10.255929208312782,
  695.3327304360063,
  -825.0005885451519],
 '3': [123.16644491718408,
  11.394409484286687,
  995.5144900045773,
  -528.7199986036221],
 '4': [120.86118215639954,
  11.872810340921404,
  1167.1489309838475,
  -402.42499916054317],
 '5': [126.9997883070342,
  16.255627883288646,
  1481.7810576085596,
  -269.4999999456819],
 '6': [119.25476285295584,
  14.835968087521573,
  1364.0065016742853,
  -269.4999701936523],
 '7': [108.69621494298353,
  17.15876323220446,
  1622.9546611452242,
  -15.600070632140845],
 '8': [109.90928432703849,
  17.567491669800084,
  1680.4553003635694,
  8.249999780811013],
 '9': [102.8409930663371,
  16.208374763244265,
  1572.6991104768415,
  8.249568110099688],
 '10': [99.08553745547157,
  15.434592012211116,
  1515.6324540577423,
  8.24999989317264],
 '11': [97.49888549425924,
  15.348179385858243,
  1491.6829539883647,
 

In [8]:
{'setpoints': {'RatioTapChanger.reg1a': {'decarbonization-app': 7, 'resilience-app': 7, 'profit_cvr-app': -4}, 'RatioTapChanger.reg2a': {'decarbonization-app': 0, 'resilience-app': -4, 'profit_cvr-app': 0}, 'RatioTapChanger.reg3a': {'decarbonization-app': 1, 'resilience-app': 0, 'profit_cvr-app': 0}, 'RatioTapChanger.reg3c': {'decarbonization-app': -5, 'resilience-app': -1, 'profit_cvr-app': -1}, 'RatioTapChanger.reg4a': {'decarbonization-app': 2, 'resilience-app': -8, 'profit_cvr-app': -1}, 'RatioTapChanger.reg4b': {'decarbonization-app': 1, 'resilience-app': -3, 'profit_cvr-app': -2}, 'RatioTapChanger.reg4c': {'decarbonization-app': -3, 'resilience-app': -4, 'profit_cvr-app': -1}, 'BatteryUnit.battery1': {'decarbonization-app': 125000.0, 'resilience-app': 125000.0, 'profit_cvr-app': 125000.0}, 'BatteryUnit.battery2': {'decarbonization-app': 200000.0, 'resilience-app': 200000.0, 'profit_cvr-app': 200000.0}, 'BatteryUnit.battery3': {'decarbonization-app': 100000.0, 'resilience-app': 100000.0, 'profit_cvr-app': 100000.0}, 'BatteryUnit.battery4': {'decarbonization-app': 150000.0, 'resilience-app': 150000.0, 'profit_cvr-app': 150000.0}, 'BatteryUnit.battery5': {'decarbonization-app': 135901.0, 'resilience-app': 250000.0, 'profit_cvr-app': 50806.425}}, 'timestamps': {'resilience-app': 41, 'profit_cvr-app': 41, 'decarbonization-app': 41}}


{'setpoints': {'RatioTapChanger.reg1a': {'decarbonization-app': 7,
   'resilience-app': 7,
   'profit_cvr-app': -4},
  'RatioTapChanger.reg2a': {'decarbonization-app': 0,
   'resilience-app': -4,
   'profit_cvr-app': 0},
  'RatioTapChanger.reg3a': {'decarbonization-app': 1,
   'resilience-app': 0,
   'profit_cvr-app': 0},
  'RatioTapChanger.reg3c': {'decarbonization-app': -5,
   'resilience-app': -1,
   'profit_cvr-app': -1},
  'RatioTapChanger.reg4a': {'decarbonization-app': 2,
   'resilience-app': -8,
   'profit_cvr-app': -1},
  'RatioTapChanger.reg4b': {'decarbonization-app': 1,
   'resilience-app': -3,
   'profit_cvr-app': -2},
  'RatioTapChanger.reg4c': {'decarbonization-app': -3,
   'resilience-app': -4,
   'profit_cvr-app': -1},
  'BatteryUnit.battery1': {'decarbonization-app': 125000.0,
   'resilience-app': 125000.0,
   'profit_cvr-app': 125000.0},
  'BatteryUnit.battery2': {'decarbonization-app': 200000.0,
   'resilience-app': 200000.0,
   'profit_cvr-app': 200000.0},
  'Batte